In [ ]:
# 여욱님 작성해주신 코드
import requests
from bs4 import BeautifulSoup as bs

url = "https://www.fmkorea.com/search.php?mid=stock&sort_index=pop&listStyle=list&search_keyword=%EC%82%BC%EC%84%B1&search_target=title_content&page=27"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url, headers=headers)
soup = bs(res.text, "lxml")

rows = soup.select("tbody > tr:not(.notice):not(.ad_wrapper)")

dates = [r.select_one("td.time").get_text(strip=True) for r in rows if r.select_one("td.time")]
titles = [r.select_one("td.title a.hx").get_text(" ", strip=True) for r in rows if r.select_one("td.title a.hx")]

print(dates[:5])    
print(titles[:5])


['2025.09.18', '2025.09.17', '2025.09.16']
['삼성 전자보면 말 그대로 대 유동성 파티의 장인거 같음 ㅋㅋㅋㅋㅋㅋㅋㅋ', '삼성 전자, 이말올의 정체 분석!!! (feat.외인 삼전 지분)', '간단한 내일 국장 시나리오입니다']


In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep

URL = "https://www.teamblind.com/kr/topics/%EC%A3%BC%EC%8B%9D%C2%B7%ED%88%AC%EC%9E%90"

driver = wb.Chrome()
driver.implicitly_wait(3)   
driver.get(URL)

sleep(2)

info = []

try:
    cards = driver.find_elements(By.CSS_SELECTOR, "div.article-list-pre")

    print("카드 개수:", len(cards))

    for card in cards[:5]:   
        try:
            title_el = card.find_element(By.CSS_SELECTOR, "div.tit a")
            title = title_el.text.strip()
        except NoSuchElementException:
            continue

        try:
            content_el = card.find_element(By.CSS_SELECTOR, "p.pre-txt")
            content = content_el.text.strip()
        except NoSuchElementException:
            content = ""

        info.append({
            "title": title,
            "content": content,
        })

    for row in info:
        print("제목:", row["title"])
        print("내용:", row["content"])
        print("-" * 30)

except Exception as e:
    print("에러:", e)
finally:
    driver.quit()


카드 개수: 69
제목: 원달러 환울 그래프 좀 웃기긴 하네
내용: 정부의 능력을 보여주겠다더니 ㅋㅋㅋㅋㅋㅋㅋ 바로 회귀 ㅋㅋㅋㅋ
------------------------------
제목: 환율 오늘 1480원 찍겠네
내용: 20조 갖다 버린거 딱 15일 갔네 연말 종가관리에 20조를 태워?
------------------------------
제목: 뉴로메카라는 회사 알아?
내용: NXT에서 상한가 틀어막았네 내일 이거 사는거 어떻게 생각해?
------------------------------
제목: 다들 보셨죠?
내용: 현금 < 예적금 < 국장 < 미장 = 서울부동산 안 하는 것보단 낫지만 할거면 미장하는것이 나았다 현금 들고 청년도약계좌 같은 개소리 하는 애들은 다 ㅈ되고 있다 https://youtube.com/shorts/8yuE_5L9qts?si=AOJY_hzAidF-6UCJ
------------------------------


In [4]:
import re
from datetime import datetime, timedelta

def parse_date_kor(text: str) -> datetime | None:
    """
    블라인드 검색 결과 날짜 문자열을 datetime으로 변환.
    예:
      - '작성시간 11분'
      - '작성시간 3시간'
      - '작성시간 1일전', '7일전'
      - '어제'
      - '01.07', '12.30'
      - '25.01.14', '2025.01.14'
    """
    raw = text
    text = text.strip()
    now = datetime.now()

    # '작성시간' 같은 접두어 제거
    text = text.replace("작성시간", "").strip()

    # ----- 1) 상대 시간 표현 처리 -----
    # 안에 들어있는 숫자 하나 뽑기
    m = re.search(r"\d+", text)
    num = int(m.group()) if m else None

    try:
        if "분" in text and num is not None:
            return now - timedelta(minutes=num)
        if "시간" in text and num is not None:
            return now - timedelta(hours=num)
        if "어제" in text:
            return now - timedelta(days=1)
        # '1일전', '7일전', '3일' 같은 것
        if "일" in text and num is not None and "." not in text:
            return now - timedelta(days=num)
    except ValueError:
        pass

    # ----- 2) 절대 날짜 표현 -----
    # 2-1) 연도가 있는 경우: '25.01.14', '2025.01.14'
    for fmt in ("%y.%m.%d", "%Y.%m.%d"):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            continue

    # 2-2) 연도 없는 '01.07' 같은 경우 → 올해 기준으로 해석,
    #      만약 오늘보다 미래 날짜면 작년으로 돌려주기
    if re.fullmatch(r"\d{2}\.\d{2}", text):
        month, day = map(int, text.split("."))
        year = now.year
        dt = datetime(year, month, day)
        if dt > now:         # 미래면 작년 날짜로 간주
            dt = datetime(year - 1, month, day)
        return dt

    # 여기까지도 안 맞으면 None 리턴
    print(f"[WARN] 날짜 해석 실패: {raw!r}")
    return None


In [11]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from datetime import datetime, timedelta
import re

# --------------------------------------------------------------------
# 0. 설정
# --------------------------------------------------------------------

URL = "https://www.teamblind.com/kr/search/%EC%82%BC%EC%84%B1"  # '삼성' 검색

# 우리가 보고 싶은 기간 (1년)
START_DATE = datetime(2025, 1, 14).date()
END_DATE   = datetime(2026, 1, 14).date()

MAX_SCROLL = 80      # 최대 스크롤 횟수 (하지만 보통 그 전에 멈출 거야)
SCROLL_PAUSE = 1.5   # 스크롤 후 대기(초)


# --------------------------------------------------------------------
# 1. 날짜 문자열 -> datetime 변환 함수
# --------------------------------------------------------------------
def parse_date_kor(text: str) -> datetime | None:
    """
    블라인드 검색 결과 날짜 문자열을 datetime으로 변환.
    예:
      - '작성시간 11분'
      - '작성시간 3시간'
      - '작성시간 1일전', '7일전'
      - '어제'
      - '01.07', '12.30'
      - '25.01.14', '2025.01.14'
      - '작성시간\n2025.12.31.'
    """
    raw = text
    text = text.strip()

    # '작성시간' 제거 + 양쪽 공백/개행 제거
    text = text.replace("작성시간", "").strip()
    # 맨 끝에 붙은 점(.) 제거 (2025.12.31. -> 2025.12.31)
    text = text.strip(" .")

    now = datetime.now()

    # 1) 상대 시간 처리
    m = re.search(r"\d+", text)
    num = int(m.group()) if m else None

    try:
        if "분" in text and num is not None:
            return now - timedelta(minutes=num)
        if "시간" in text and num is not None:
            return now - timedelta(hours=num)
        if "어제" in text:
            return now - timedelta(days=1)
        if "일" in text and "." not in text and num is not None:
            # '1일전', '7일전', '3일' 등
            return now - timedelta(days=num)
    except ValueError:
        pass

    # 2) 절대 날짜 처리 ('25.01.14', '2025.01.14')
    for fmt in ("%y.%m.%d", "%Y.%m.%d"):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            continue

    # 3) 월.일 ('01.07', '12.30')
    if re.fullmatch(r"\d{2}\.\d{2}", text):
        month, day = map(int, text.split("."))
        year = now.year
        dt = datetime(year, month, day)
        if dt.date() > now.date():
            dt = datetime(year - 1, month, day)
        return dt

    print(f"[WARN] 날짜 해석 실패: {raw!r}")
    return None



# --------------------------------------------------------------------
# 2. 메인 크롤링
# --------------------------------------------------------------------
driver = wb.Chrome()
driver.implicitly_wait(3)
driver.get(URL)

wait = WebDriverWait(driver, 10)
sleep(2)

# 2-1. 필터: 주식·투자 + 최신순
try:
    # 카테고리 드롭다운 (전체(****)) 클릭
    category_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'전체') and contains(text(),'(')]")
        )
    )
    category_button.click()
    sleep(1)

    # '주식·투자' 선택
    stock_item = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'주식·투자')]")
        )
    )
    stock_item.click()
    sleep(2)

    # 정렬 드롭다운 (최신순/추천순)
    sort_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'최신순') or contains(text(),'추천순')]")
        )
    )
    sort_button.click()
    sleep(1)

    # '최신순' 선택
    latest_item = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'최신순')]")
        )
    )
    latest_item.click()
    sleep(2)

    print("✅ 주식·투자 + 최신순 선택 완료")
except Exception as e:
    print("필터 선택 중 에러:", e)

# 2-2. 카드 돌면서 1년치 글 개수 세기
total_in_range = 0
seen_keys = set()   # (제목, 날짜문자열)로 중복 방지
stop_here = False   # START_DATE 보다 옛날 글 나오면 True로 바꿔서 종료

try:
    for i in range(MAX_SCROLL):
        cards = driver.find_elements(By.CSS_SELECTOR, "div.article-list-pre")
        print(f"[스크롤 {i+1}] 카드 개수: {len(cards)}")

        for card in cards:
            # 제목
            try:
                title_el = card.find_element(By.CSS_SELECTOR, "div.tit a")
                title = title_el.text.strip()
            except NoSuchElementException:
                continue

            # 날짜
            try:
                date_el = card.find_element(By.CSS_SELECTOR, "a.past")
                date_text = date_el.text.strip()
            except NoSuchElementException:
                continue

            key = (title, date_text)
            if key in seen_keys:
                continue
            seen_keys.add(key)

            dt = parse_date_kor(date_text)
            if dt is None:
                continue

            d = dt.date()

            # 👉 여기서 바로 범위 체크
            # 1) START_DATE보다 옛날 글이 나오면: 이제 더 이상 볼 필요 없음 → 전체 종료
            if d < START_DATE:
                stop_here = True
                print(f"  → {d} 가 {START_DATE}보다 이전이라, 여기서 중단할게요.")
                break

            # 2) 1년 범위 안에 있으면 카운트 +
            if START_DATE <= d <= END_DATE:
                total_in_range += 1

        print(f"  → 지금까지 1년 범위 글 개수: {total_in_range}")

        if stop_here:
            print("1년 시작 날짜보다 옛날 글이 나왔으니 더 이상 스크롤하지 않습니다.")
            break

        # 다음 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(SCROLL_PAUSE)

    print("=" * 50)
    print(f"최종 1년치 글 개수: {total_in_range}")

except Exception as e:
    print("에러:", e)

finally:
    driver.quit()

✅ 주식·투자 + 최신순 선택 완료
[스크롤 1] 카드 개수: 27
  → 지금까지 1년 범위 글 개수: 17
[스크롤 2] 카드 개수: 54
  → 지금까지 1년 범위 글 개수: 37
[스크롤 3] 카드 개수: 81
  → 지금까지 1년 범위 글 개수: 57
[스크롤 4] 카드 개수: 161
  → 지금까지 1년 범위 글 개수: 116
[스크롤 5] 카드 개수: 187
  → 지금까지 1년 범위 글 개수: 136
[스크롤 6] 카드 개수: 214
  → 지금까지 1년 범위 글 개수: 155
[스크롤 7] 카드 개수: 241
  → 지금까지 1년 범위 글 개수: 175
[스크롤 8] 카드 개수: 267
  → 지금까지 1년 범위 글 개수: 195
[스크롤 9] 카드 개수: 374
  → 지금까지 1년 범위 글 개수: 275
[스크롤 10] 카드 개수: 454
  → 지금까지 1년 범위 글 개수: 335
[스크롤 11] 카드 개수: 1041
  → 지금까지 1년 범위 글 개수: 773
[스크롤 12] 카드 개수: 1254
  → 2025-01-13 가 2025-01-14보다 이전이라, 여기서 중단할게요.
  → 지금까지 1년 범위 글 개수: 858
1년 시작 날짜보다 옛날 글이 나왔으니 더 이상 스크롤하지 않습니다.
최종 1년치 글 개수: 858


In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from datetime import datetime, timedelta
import re
import csv

# --------------------------------------------------------------------
# 0. 설정
# --------------------------------------------------------------------

# '삼성' 검색 결과 페이지
URL = "https://www.teamblind.com/kr/search/%EC%82%BC%EC%84%B1"

# 보고 싶은 기간: 2025-01-14 ~ 2026-01-14
START_DATE = datetime(2025, 1, 14).date()
END_DATE   = datetime(2026, 1, 14).date()

# 스크롤 관련 설정
MAX_SCROLL = 80      # 최대 스크롤 횟수
SCROLL_PAUSE = 1.5   # 스크롤 후 기다리는 시간(초)

# 저장할 CSV 파일 이름
CSV_PATH = "blind_samsung_1year.csv"


# --------------------------------------------------------------------
# 1. 블라인드 날짜 문자열 -> datetime 변환 함수
# --------------------------------------------------------------------
def parse_date_kor(text: str) -> datetime | None:
    """
    블라인드 검색 결과 날짜 문자열을 datetime으로 변환.

    예:
      '작성시간 11분'
      '작성시간 3시간'
      '작성시간 1일전', '7일전'
      '어제'
      '01.07', '12.30'
      '25.01.14', '2025.01.14'
      '작성시간\\n2025.12.31.'
    """
    raw = text
    text = text.strip()

    # '작성시간' 제거 + 양쪽 공백/개행 제거
    text = text.replace("작성시간", "").strip()
    # 맨 끝에 붙은 점(.) 제거 (2025.12.31. -> 2025.12.31)
    text = text.strip(" .")

    now = datetime.now()

    # ----- 1) 상대 시간 처리 -----
    m = re.search(r"\d+", text)
    num = int(m.group()) if m else None

    try:
        if "분" in text and num is not None:
            return now - timedelta(minutes=num)
        if "시간" in text and num is not None:
            return now - timedelta(hours=num)
        if "어제" in text:
            return now - timedelta(days=1)
        if "일" in text and "." not in text and num is not None:
            # '1일전', '7일전', '3일' 등
            return now - timedelta(days=num)
    except ValueError:
        pass

    # ----- 2) 절대 날짜 처리 ('25.01.14', '2025.01.14') -----
    for fmt in ("%y.%m.%d", "%Y.%m.%d"):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            continue

    # ----- 3) 월.일 ('01.07', '12.30') -----
    if re.fullmatch(r"\d{2}\.\d{2}", text):
        month, day = map(int, text.split("."))
        year = now.year
        dt = datetime(year, month, day)
        # 오늘보다 미래 날짜면 작년 것으로 간주
        if dt.date() > now.date():
            dt = datetime(year - 1, month, day)
        return dt

    print(f"[WARN] 날짜 해석 실패: {raw!r}")
    return None


# --------------------------------------------------------------------
# 2. 메인 크롤링 로직
# --------------------------------------------------------------------
driver = wb.Chrome()
driver.implicitly_wait(3)
driver.get(URL)

wait = WebDriverWait(driver, 10)
sleep(2)

# 2-1. 필터: 주식·투자 + 최신순
try:
    # 카테고리 드롭다운 (전체(****)) 클릭
    category_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'전체') and contains(text(),'(')]")
        )
    )
    category_button.click()
    sleep(1)

    # '주식·투자' 선택
    stock_item = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'주식·투자')]")
        )
    )
    stock_item.click()
    sleep(2)

    # 정렬 드롭다운 (최신순/추천순)
    sort_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'최신순') or contains(text(),'추천순')]")
        )
    )
    sort_button.click()
    sleep(1)

    # '최신순' 선택
    latest_item = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'최신순')]")
        )
    )
    latest_item.click()
    sleep(2)

    print("✅ 주식·투자 + 최신순 선택 완료")
except Exception as e:
    print("필터 선택 중 에러:", e)

# 1년 범위 글 정보 저장용 리스트
posts = []

# 2-2. 스크롤하면서 1년치 글들 수집
total_in_range = 0          # 1년 범위 안 글 개수
stop_here = False           # 시작 날짜보다 옛날 글 나오면 True
prev_count = 0              # 직전까지 본 카드 개수

try:
    for i in range(MAX_SCROLL):
        cards = driver.find_elements(By.CSS_SELECTOR, "div.article-list-pre")
        cur_count = len(cards)
        print(f"[스크롤 {i+1}] 카드 개수: {cur_count}")

        # 새로 생긴 카드만 처리 (이미 본 건 다시 안 봄)
        new_cards = cards[prev_count:cur_count]
        print(f"  → 이번 스크롤에서 새로 나온 카드: {len(new_cards)}")

        if not new_cards:
            print("  → 새 카드가 더 이상 없어서 여기서 중단할게요.")
            break

        for card in new_cards:
            # 제목 & URL
            try:
                title_el = card.find_element(By.CSS_SELECTOR, "div.tit a")
                title = title_el.text.strip()
                url = title_el.get_attribute("href")
            except NoSuchElementException:
                continue

            # 날짜 (작성시간 11분 / 2025.12.31. 등)
            try:
                date_el = card.find_element(By.CSS_SELECTOR, "a.past")
                date_text = date_el.text.strip()
            except NoSuchElementException:
                continue

            dt = parse_date_kor(date_text)
            if dt is None:
                continue
            d = dt.date()

            # 1) 시작 날짜보다 옛날 글 → 더 이상 볼 필요 없음
            if d < START_DATE:
                stop_here = True
                print(f"  → {d} 가 {START_DATE}보다 이전이라, 여기서 중단할게요.")
                break

            # 2) 1년 범위 안이 아니면 skip
            if not (START_DATE <= d <= END_DATE):
                continue

            # 여기까지 왔으면 1년 범위 안 글
            total_in_range += 1

            # ---- 조회수 / 좋아요 / 댓글수 추출 ----
            views = likes = comments = ""

            try:
                # info_fnc 안에 아이콘+숫자들이 span에 들어있다고 가정
                # 예: [조회수, 좋아요, 댓글]
                stat_spans = card.find_elements(By.CSS_SELECTOR, "div.info_fnc span")
                if len(stat_spans) >= 3:
                    views = stat_spans[0].text.strip()
                    likes = stat_spans[1].text.strip()
                    comments = stat_spans[2].text.strip()
            except NoSuchElementException:
                pass

            # ---- 요약 내용(리스트에 보이는 한 줄) ----
            try:
                content_el = card.find_element(By.CSS_SELECTOR, "p.pre-txt")
                content = content_el.text.strip()
            except NoSuchElementException:
                content = ""

            posts.append({
                "title": title,
                "date": d.isoformat(),
                "views": views,
                "likes": likes,
                "comments": comments,
                "content": content
            })

        print(f"  → 지금까지 1년 범위 글 개수: {total_in_range}")

        if stop_here:
            print("1년 시작 날짜보다 옛날 글이 나왔으니 더 이상 스크롤하지 않습니다.")
            break

        # 다음 스크롤 대비해서 기준 업데이트
        prev_count = cur_count

        # 다음 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(SCROLL_PAUSE)

    print("=" * 50)
    print(f"최종 1년치 글 개수(카운트 기준): {total_in_range}")
    print(f"수집된 글 row 수: {len(posts)}")

except Exception as e:
    print("에러:", e)

finally:
    driver.quit()


# --------------------------------------------------------------------
# 3. CSV로 저장
# --------------------------------------------------------------------
if posts:
    fieldnames = ["title", "date", "views", "likes", "comments", "content"]
    with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(posts)

    print(f"CSV 저장 완료: {CSV_PATH} (행 수: {len(posts)})")
else:
    print("수집된 글이 없어 CSV를 만들지 않았습니다.")


✅ 주식·투자 + 최신순 선택 완료
[스크롤 1] 카드 개수: 27
  → 이번 스크롤에서 새로 나온 카드: 27
  → 지금까지 1년 범위 글 개수: 17
[스크롤 2] 카드 개수: 374
  → 이번 스크롤에서 새로 나온 카드: 347
에러: HTTPConnectionPool(host='localhost', port=3590): Read timed out. (read timeout=120)
CSV 저장 완료: blind_samsung_1year.csv (행 수: 17)


In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from datetime import datetime, timedelta
import re

# --------------------------------------------------------------------
# 0. 설정
# --------------------------------------------------------------------

URL = "https://www.teamblind.com/kr/search/%EC%82%BC%EC%84%B1"  # '삼성' 검색 결과

# 스크롤을 멈추고 싶은 기준 날짜
START_DATE = datetime(2025, 1, 14).date()

MAX_SCROLL = 40       # 절대 이 이상은 안 감 (노트북 안전장치)
SCROLL_PAUSE = 1.5    # 스크롤 후 잠깐 대기


# --------------------------------------------------------------------
# 1. 날짜 문자열 파싱
# --------------------------------------------------------------------
def parse_date_kor(text: str) -> datetime | None:
    """
    블라인드 검색 결과 날짜 문자열을 datetime으로 변환.

    예:
      '작성시간 11분'
      '작성시간 3시간'
      '작성시간 1일전', '7일전'
      '3달전', '2개월전', '1년전'
      '어제'
      '01.07', '12.30'
      '25.01.14', '2025.01.14'
      '작성시간\\n2025.12.31.'
    """
    raw = text
    text = text.strip()

    # '작성시간' 제거 + 맨 끝 점 제거
    text = text.replace("작성시간", "").strip()
    text = text.strip(" .")

    now = datetime.now()

    # 숫자 추출
    m = re.search(r"\d+", text)
    num = int(m.group()) if m else None

    # ---- 상대 시간 처리 ----
    try:
        if "분" in text and num is not None:
            return now - timedelta(minutes=num)
        if "시간" in text and num is not None:
            return now - timedelta(hours=num)
        if "어제" in text:
            return now - timedelta(days=1)
        if "일" in text and "." not in text and num is not None:
            # '1일전', '7일전', '3일' 등
            return now - timedelta(days=num)
        if ("달" in text or "개월" in text) and num is not None:
            # 대충 30일 기준으로 개월수
            return now - timedelta(days=30 * num)
        if "년" in text and num is not None:
            # 대충 365일 기준으로 년수
            return now - timedelta(days=365 * num)
    except ValueError:
        pass

    # ---- 절대 날짜 처리 ----
    # '25.01.14', '2025.01.14'
    for fmt in ("%y.%m.%d", "%Y.%m.%d"):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            continue

    # '01.07', '12.30'
    if re.fullmatch(r"\d{2}\.\d{2}", text):
        month, day = map(int, text.split("."))
        year = now.year
        dt = datetime(year, month, day)
        if dt.date() > now.date():
            dt = datetime(year - 1, month, day)
        return dt

    print(f"[WARN] 날짜 해석 실패: {raw!r}")
    return None


# --------------------------------------------------------------------
# 2. 메인 로직: 2025-01-14 근처까지 안전하게 스크롤
# --------------------------------------------------------------------
options = wb.ChromeOptions()
# options.add_argument("--headless=new")  # 화면 안 보이게 돌리고 싶으면 켜기

driver = wb.Chrome(options=options)
driver.implicitly_wait(3)
driver.get(URL)

wait = WebDriverWait(driver, 10)
sleep(2)

# 2-1. 필터: 주식·투자 + 최신순
try:
    category_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'전체') and contains(text(),'(')]")
        )
    )
    category_button.click()
    sleep(1)

    stock_item = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'주식·투자')]")
        )
    )
    stock_item.click()
    sleep(2)

    sort_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'최신순') or contains(text(),'추천순')]")
        )
    )
    sort_button.click()
    sleep(1)

    latest_item = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//*[contains(text(),'최신순')]")
        )
    )
    latest_item.click()
    sleep(2)

    print("✅ 주식·투자 + 최신순 선택 완료")
except Exception as e:
    print("필터 선택 중 에러:", e)

oldest_seen: datetime | None = None

try:
    for i in range(MAX_SCROLL):
        cards = driver.find_elements(By.CSS_SELECTOR, "div.article-list-pre")
        print(f"[스크롤 {i+1}] 현재 카드 개수: {len(cards)}")

        # 이 스크롤에서 본 카드들의 최소 날짜
        local_oldest = None

        for card in cards:
            try:
                date_el = card.find_element(By.CSS_SELECTOR, "a.past")
                date_text = date_el.text.strip()
            except NoSuchElementException:
                continue

            dt = parse_date_kor(date_text)
            if dt is None:
                continue
            d = dt.date()

            # 현재 스크롤에서 가장 오래된 날짜(local_oldest)
            if local_oldest is None or d < local_oldest:
                local_oldest = d

            # 전체 기준에서 가장 오래된 날짜(oldest_seen)
            if oldest_seen is None or d < oldest_seen:
                oldest_seen = d

        print(f"  → 이번 스크롤에서 가장 오래된 날짜: {local_oldest}")
        print(f"  → 지금까지 본 것 중 가장 오래된 날짜: {oldest_seen}")

        # ✅ 여기서 브레이크: 2025-01-14보다 옛날 날짜를 발견하면 멈춤
        if oldest_seen is not None and oldest_seen < START_DATE:
            print(f"🛑 {oldest_seen} 이(가) {START_DATE}보다 이전이라, 여기서 스크롤 중단!")
            break

        # 다음 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(SCROLL_PAUSE)

    print("=== 스크롤 종료 ===")
    print("최종 oldest_seen:", oldest_seen)

except Exception as e:
    print("에러:", e)

finally:
    driver.quit()


✅ 주식·투자 + 최신순 선택 완료
[스크롤 1] 현재 카드 개수: 27
  → 이번 스크롤에서 가장 오래된 날짜: 2026-01-12
  → 지금까지 본 것 중 가장 오래된 날짜: 2026-01-12
[스크롤 2] 현재 카드 개수: 241
  → 이번 스크롤에서 가장 오래된 날짜: 2025-11-10
  → 지금까지 본 것 중 가장 오래된 날짜: 2025-11-10
[스크롤 3] 현재 카드 개수: 1947
에러: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=143.0.7499.193); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7c6b088d5
	0x7ff7c6b08930
	0x7ff7c68e165d
	0x7ff7c68e92cc
	0x7ff7c68ec3b4
	0x7ff7c68ec47f
	0x7ff7c69392e5
	0x7ff7c6939d3c
	0x7ff7c692c88c
	0x7ff7c692c746
	0x7ff7c698ac97
	0x7ff7c692ac29
	0x7ff7c692ba93
	0x7ff7c6e20620
	0x7ff7c6e1af60
	0x7ff7c6e396c6
	0x7ff7c6b25dd4
	0x7ff7c6b2ed7c
	0x7ff7c6b11ff4
	0x7ff7c6b121a5
	0x7ff7c6af7ed2
	0x7ffebb237374
	0x7ffebc53cc91

